In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/amexfeather/test_data_f32.ftr
/kaggle/input/amexfeather/train_data.ftr
/kaggle/input/amexfeather/train_data_f32.ftr
/kaggle/input/amexfeather/test_data.ftr
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [4]:
train_dataset_ = pd.read_feather('../input/amexfeather/train_data.ftr')
# Keep the latest statement features for each customer
train_dataset = train_dataset_.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()

In [5]:
del train_dataset_
gc.collect()

23

In [6]:
for col in train_dataset.columns:
    if train_dataset[col].dtype == "category":
        print(col)

D_63
D_64
D_66
D_68
B_30
B_38
D_114
D_116
D_117
D_120
D_126


In [7]:
categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

num_cols = [col for col in train_dataset.columns if col not in categorical_cols + ["target"]]

print(f'Total features = {1}')
print(f'Total categorical features =  {len(categorical_cols)}')
print(f'Total number of continuos features = {len(num_cols)}')

Total features = 1
Total categorical features =  11
Total number of continuos features = 178


In [8]:
for col in train_dataset.columns:
    print(col," - ", (train_dataset[col].isna().sum())*100/ train_dataset.shape[0])


S_2  -  0.0
P_2  -  0.6469635856905339
D_39  -  0.0
B_1  -  0.0
B_2  -  0.006755093013272668
R_1  -  0.0
S_3  -  18.51549204315415
D_41  -  0.006755093013272668
B_3  -  0.006755093013272668
D_42  -  86.94523798628498
D_43  -  29.269600120284238
D_44  -  4.858219313900456
B_4  -  0.0
D_45  -  0.006755093013272668
B_5  -  0.0
R_2  -  0.0
D_46  -  20.727893958114066
D_47  -  0.0
D_48  -  12.636817871797051
D_49  -  88.72052001141829
B_6  -  0.008716249049384088
B_7  -  0.0
B_8  -  0.8914543715257576
D_50  -  57.142639236630906
D_51  -  0.0
B_9  -  0.0
R_3  -  0.0
D_52  -  0.27020372053090674
P_3  -  4.841876346932861
B_10  -  0.0
D_53  -  71.02261212909636
S_5  -  0.0
B_11  -  0.0
S_6  -  0.0
D_54  -  0.006755093013272668
R_4  -  0.0
S_7  -  18.51549204315415
B_12  -  0.0
S_8  -  0.0
D_55  -  6.619337434328511
D_56  -  53.32906237129914
B_13  -  0.34058743160468324
R_5  -  0.0
D_58  -  0.0
S_9  -  40.063802943041495
B_14  -  0.0
D_59  -  0.8903648403945846
D_60  -  0.0
D_61  -  10.5353302

In [9]:
for col in train_dataset.columns:
    if train_dataset[col].isna().sum() > train_dataset.shape[0]*0.75:
        print(col)

D_42
D_49
D_66
D_73
D_76
R_9
B_29
D_87
D_88
D_106
R_26
D_108
D_110
D_111
B_39
B_42
D_132
D_134
D_135
D_136
D_137
D_138
D_142


In [10]:
# Dropping features which have null values more than 75%
train_dataset = train_dataset.drop(['S_2','D_66','D_42','D_49','D_73','D_76','R_9','B_29','D_87','D_88','D_106','R_26','D_108','D_110','D_111','B_39','B_42','D_132','D_134','D_135','D_136','D_137','D_138','D_142'], axis=1)

In [11]:
print(num_cols)

['S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_

In [12]:
selected_col = np.array(['P_2','S_3','B_2','D_41','D_43','B_3','D_44','D_45','D_46','D_48','D_50','D_53','S_7','D_56','S_9','B_6','B_8','D_52','P_3','D_54','D_55','B_13','D_59','D_61','B_15','D_62','B_16','B_17','D_77','B_19','B_20','D_69','B_22','D_70','D_72','D_74','R_7','B_25','B_26','D_78','D_79','D_80','B_27','D_81','R_12','D_82','D_105','S_27','D_83','R_14','D_84','D_86','R_20','B_33','D_89','D_91','S_22','S_23','S_24','S_25','S_26','D_102','D_103','D_104','D_107','B_37','R_27','D_109','D_112','B_40','D_113','D_115','D_118','D_119','D_121','D_122','D_123','D_124','D_125','D_128','D_129','B_41','D_130','D_131','D_133','D_139','D_140','D_141','D_143','D_144','D_145'])

for col in selected_col:
    train_dataset[col] = train_dataset[col].fillna(train_dataset[col].median())

In [13]:
selcted_col2 = np.array(['D_68','B_30','B_38','D_64','D_114','D_116','D_117','D_120','D_126'])

for col2 in selcted_col2:
    train_dataset[col2] =  train_dataset[col2].fillna(train_dataset[col2].mode()[0])

In [14]:
for col in train_dataset.columns:
    print(col," - ", (train_dataset[col].isna().sum())*100/ train_dataset.shape[0])


P_2  -  0.0
D_39  -  0.0
B_1  -  0.0
B_2  -  0.0
R_1  -  0.0
S_3  -  0.0
D_41  -  0.0
B_3  -  0.0
D_43  -  0.0
D_44  -  0.0
B_4  -  0.0
D_45  -  0.0
B_5  -  0.0
R_2  -  0.0
D_46  -  0.0
D_47  -  0.0
D_48  -  0.0
B_6  -  0.0
B_7  -  0.0
B_8  -  0.0
D_50  -  0.0
D_51  -  0.0
B_9  -  0.0
R_3  -  0.0
D_52  -  0.0
P_3  -  0.0
B_10  -  0.0
D_53  -  0.0
S_5  -  0.0
B_11  -  0.0
S_6  -  0.0
D_54  -  0.0
R_4  -  0.0
S_7  -  0.0
B_12  -  0.0
S_8  -  0.0
D_55  -  0.0
D_56  -  0.0
B_13  -  0.0
R_5  -  0.0
D_58  -  0.0
S_9  -  0.0
B_14  -  0.0
D_59  -  0.0
D_60  -  0.0
D_61  -  0.0
B_15  -  0.0
S_11  -  0.0
D_62  -  0.0
D_63  -  0.0
D_64  -  0.0
D_65  -  0.0
B_16  -  0.0
B_17  -  0.0
B_18  -  0.0
B_19  -  0.0
B_20  -  0.0
D_68  -  0.0
S_12  -  0.0
R_6  -  0.0
S_13  -  0.0
B_21  -  0.0
D_69  -  0.0
B_22  -  0.0
D_70  -  0.0
D_71  -  0.0
D_72  -  0.0
S_15  -  0.0
B_23  -  0.0
P_4  -  0.0
D_74  -  0.0
D_75  -  0.0
B_24  -  0.0
R_7  -  0.0
D_77  -  0.0
B_25  -  0.0
B_26  -  0.0
D_78  -  0.0
D_79  -  0.

In [15]:
test_dataset_ = pd.read_feather('../input/amexfeather/test_data.ftr')
# Keep the latest statement features for each customer
test_dataset = test_dataset_.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()

In [16]:
del test_dataset_
gc.collect()

23

In [17]:
test_dataset = test_dataset.drop(['S_2','D_42','D_49','D_66','D_73','D_76','R_9','B_29','D_87','D_88','D_106','R_26','D_108','D_110','D_111','B_39','B_42','D_132','D_134','D_135','D_136','D_137','D_138','D_142'], axis=1)

In [18]:
selected_column = np.array(['P_2','S_3','B_2','D_41','D_43','B_3','D_44','D_45','D_46','D_48','D_50','D_53','S_7','D_56','S_9','S_12','S_17','B_6','B_8','D_52','P_3','D_54','D_55','B_13','D_59','D_61','B_15','D_62','B_16','B_17','D_77','B_19','B_20','D_69','B_22','D_70','D_72','D_74','R_7','B_25','B_26','D_78','D_79','D_80','B_27','D_81','R_12','D_82','D_105','S_27','D_83','R_14','D_84','D_86','R_20','B_33','D_89','D_91','S_22','S_23','S_24','S_25','S_26','D_102','D_103','D_104','D_107','B_37','R_27','D_109','D_112','B_40','D_113','D_115','D_118','D_119','D_121','D_122','D_123','D_124','D_125','D_128','D_129','B_41','D_130','D_131','D_133','D_139','D_140','D_141','D_143','D_144','D_145'])

for column in selected_column:
    test_dataset[column] = test_dataset[column].fillna(train_dataset[column].median())

In [19]:
selected_column2 = np.array(['D_68','B_30','B_38','D_114','D_116','D_117','D_120','D_126'])

for column2 in selected_column2:
    test_dataset[column2] =  test_dataset[column2].fillna(train_dataset[column2].mode()[0])

In [20]:
#  D_66 feature is removed due to having more than 75% of null values
categorical_cols.remove('D_66')

In [21]:
print(test_dataset.shape)
print(train_dataset.shape)

(924621, 165)
(458913, 166)


In [22]:
train_dataset_without_target = train_dataset.drop(["target"],axis=1)

cor_matrix = train_dataset_without_target.corr()
col_core = set()

# Removing the features with more than 90% correlation
for i in range(len(cor_matrix.columns)):
    for j in range(i):
        if(cor_matrix.iloc[i, j] > 0.9):
            col_name = cor_matrix.columns[i]
            col_core.add(col_name)
col_core

{'B_11',
 'B_13',
 'B_15',
 'B_23',
 'B_33',
 'B_37',
 'D_104',
 'D_119',
 'D_141',
 'D_143',
 'D_74',
 'D_75',
 'D_77',
 'S_24',
 'S_7'}

In [23]:
train_dataset = train_dataset.drop(col_core, axis=1)
test_dataset = test_dataset.drop(col_core, axis=1)

In [24]:
print(test_dataset.shape)
print(train_dataset.shape)

(924621, 150)
(458913, 151)


In [25]:
y = train_dataset['target']
train_dataset = train_dataset.drop(['target'],1)

In [26]:
print(train_dataset.shape)
print(y.shape)

(458913, 150)
(458913,)


In [27]:
categorical_cols=["D_63","D_64","D_68","B_30","B_38","D_114","D_116","D_117","D_120","D_126"]
# Encoding categorical columns
train_dataset = pd.get_dummies(train_dataset, columns = categorical_cols)
test_dataset = pd.get_dummies(test_dataset, columns = categorical_cols)

test_dataset = test_dataset.reindex(columns = train_dataset.columns, fill_value=0)

In [28]:
test_dataset.shape

(924621, 184)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split( train_dataset, y, test_size=0.3, random_state=68)

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
import xgboost as xgb

In [ ]:
final_model = xgb.XGBClassifier(base_score=0.5, max_depth=6,booster='gbtree',n_estimators=10,objective = 'reg:logistic', seed = 10, eta = 0.01)

In [ ]:
final_model.fit(train_dataset, y)

In [ ]:
p = final_model.predict_proba(X_test)

In [ ]:
p_v1 = p[:,1].reshape(-1,1)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

roc_auc_score(y_test, p_v1)

In [ ]:
predictions = final_model.predict_proba(test_dataset)
predictions

In [ ]:
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVC
# clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [ ]:
# clf.fit(train_dataset, y)

In [ ]:
# predsSVM = clf.predict_proba(X_test)

In [ ]:
predictions = predictions[:,1]

In [ ]:
predictions[:10]

In [ ]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': predictions})
output.to_csv('Submission v3 XGB.csv', index=False)

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()


In [ ]:
clf.fit(X_train, y_train)

In [ ]:
p_light = clf.predict_proba(X_test)

In [ ]:
p_v2 = p_light[:,1].reshape(-1,1)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc_auc_score(y_test, p_v2)

In [ ]:
predictions_light = clf.predict_proba(test_dataset)
predictions_light

In [ ]:
predictions_light = predictions_light[:,1]

In [ ]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': predictions_light})
output.to_csv('Submission v4 LGBM.csv', index=False)

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
train_dataset, y = oversample.fit_resample(train_dataset, y)

In [ ]:
clf.fit(train_dataset, y)

In [ ]:
p_light = clf.predict_proba(X_test)
predictions_light = clf.predict_proba(test_dataset)

In [ ]:
predictions_light = predictions_light[:,1]

In [ ]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': predictions_light})
output.to_csv('Submission v4 LGBM full.csv', index=False)

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [34]:
from sklearn import svm

In [35]:
clf = svm.SVC()

In [ ]:
clf.fit(X_train, y_train)